In [1]:
import pandas as pd
import numpy as np
import json
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import requests
import re

In [2]:
# Windows users
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 94.0.4606
[WDM] - Get LATEST driver version for 94.0.4606


[WDM] - Get LATEST driver version for 94.0.4606
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/94.0.4606.61/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\ajgli\.wdm\drivers\chromedriver\win32\94.0.4606.61]


In [3]:
mvp_url = 'https://www.baseball-reference.com/awards/mvp.shtml'
war_url = 'https://www.espn.com/mlb/war/leaders/_/type/seasonal/year/'

In [4]:
df = pd.read_html(mvp_url)

In [5]:
mvp_list = pd.DataFrame(df[0])

In [6]:
mvp_year = np.array(mvp_list[( 'Unnamed: 0_level_0', 'Year')].values)
mvp_name = np.array(mvp_list[( 'Unnamed: 2_level_0', 'Name')].values)
mvp_team = np.array(mvp_list[( 'Unnamed: 3_level_0', 'Tm')].values)
mvp_WAR =  np.array(mvp_list[( 'Unnamed: 4_level_0', 'WAR')].values)

In [7]:
data = {'Year Won': mvp_year, 'Name': mvp_name, 'Team': mvp_team, 'WAR': mvp_WAR}

In [8]:
mvp_df = pd.DataFrame(data)
mvp_df = mvp_df.dropna()

In [9]:
mvp_df['Year Won'] = mvp_df['Year Won'].astype(str)
mvp_df['Year Won']= mvp_df['Year Won'].str[:4]
mvp_df['Year Won'] = mvp_df['Year Won'].astype(int)

In [11]:
next_year = []
for year in mvp_df['Year Won']:
    next_year.append(year + 1)
mvp_df['Next Year'] = next_year

In [ ]:
new_df = mvp_df.loc[mvp_df['Year Won']>=1998]

In [45]:
next_war = []
for row in new_df.index:
    i = 51
    year = mvp_df['Next Year'][row]
    name = mvp_df['Name'][row]
    year_url = f'{war_url}{year}'
    war = ''
    print(f'{name} {year}')
    while i <= 201:
    #Can be while true?
        year_html = pd.read_html(year_url)
        year_df = pd.DataFrame(year_html[0])
        if year_df.loc[year_df[1] == name][2].empty and i != 151:
            year_url = f'https://www.espn.com/mlb/war/leaders/_/year/{year}/type/seasonal/alltime/false/count/{i}'
            i= i + 50
            print(f'{year} {i}')
        elif year_df.loc[year_df[1] == name][2].empty and i == 151:
            next_war.append(war)
            print(f'{year} {year}')
            break
        else:
            war = year_df.loc[year_df[1] == name][2].values[0]
            next_war.append(war)
            break
    
    
    

José Abreu 2021
2021 101
2021 151
2021 2021
Freddie Freeman 2021
Mike Trout 2020
2020 101
Cody Bellinger 2020
2020 101
Mookie Betts 2019
Christian Yelich 2019
Jose Altuve 2018
Giancarlo Stanton 2018
2018 101
Mike Trout 2017
Kris Bryant 2017
Josh Donaldson 2016
Bryce Harper 2016
2016 101
2016 151
2016 2016
Mike Trout 2015
Clayton Kershaw 2015
Miguel Cabrera 2014
Andrew McCutchen 2014
Miguel Cabrera 2013
Buster Posey 2013
Justin Verlander 2012
Ryan Braun 2012
Josh Hamilton 2011
2011 101
Joey Votto 2011
Joe Mauer 2010
Albert Pujols 2010
Dustin Pedroia 2009
Albert Pujols 2009
Alex Rodriguez 2008
Jimmy Rollins 2008
Justin Morneau 2007
2007 101
2007 151
Ryan Howard 2007
2007 101
2007 151
Alex Rodriguez 2006
2006 101
Albert Pujols 2006
Vladimir Guerrero 2005
Barry Bonds 2005
2005 101
2005 151
2005 2005
Alex Rodriguez 2004
Barry Bonds 2004
Miguel Tejada 2003
Barry Bonds 2003
Ichiro Suzuki 2002
2002 101
2002 151
Barry Bonds 2002
Jason Giambi 2001
Jeff Kent 2001
Ivan Rodriguez 2000
2000 101
2000

In [46]:
next_war

['',
 '4.5',
 '1.6',
 '1.4',
 '6.8',
 '7.1',
 '5.2',
 '4',
 '6.7',
 '6.1',
 '7.4',
 '',
 '9.4',
 '7.6',
 '4.9',
 '6.4',
 '7.2',
 '5.2',
 '7.7',
 '7',
 '3.6',
 '6.4',
 '5.7',
 '7.5',
 '5.6',
 '9.7',
 '6.8',
 '5.4',
 '2.9',
 '2.9',
 '4.5',
 '8.5',
 '5.7',
 '',
 '7.6',
 '10.7',
 '4.9',
 '9.2',
 '3.5',
 '11.8',
 '9.1',
 '5.2',
 '',
 '5.6',
 '',
 '4.9']

In [48]:
new_df['Next War'] = next_war
new_df

,Year Won,Name,Team,WAR,Next Year,Next War
0,2020,José Abreu,CHW,3.0,2021,
1,2020,Freddie Freeman,ATL,3.3,2021,4.5
3,2019,Mike Trout,LAA,7.9,2020,1.6
4,2019,Cody Bellinger,LAD,8.6,2020,1.4
6,2018,Mookie Betts,BOS,10.7,2019,6.8
7,2018,Christian Yelich,MIL,7.3,2019,7.1
9,2017,Jose Altuve,HOU,7.7,2018,5.2
10,2017,Giancarlo Stanton,MIA,7.9,2018,4
12,2016,Mike Trout,LAA,10.5,2017,6.7
13,2016,Kris Bryant,CHC,7.3,2017,6.1


In [262]:
#This is my testing area
# i = 51
# year = 2020
# year_url = f'{war_url}{year}'
# while i <= 201:
#     year_html = pd.read_html(year_url)
#     year_df = pd.DataFrame(year_html[0])
#     if year_df.loc[year_df[1] == 'Spencer Turnbull'][2].empty:
#         year_url = f'https://www.espn.com/mlb/war/leaders/_/year/{year}/type/seasonal/alltime/false/count/{i}'
#         i= i + 50
#         print('no')
#     else:
#         print(year_df.loc[year_df[1] == 'Spencer Turnbull'][2].values[0])
#         break
        
        

no
no
no
1.1
